# trustLevel_1_2

**Idea:** Train separate models based on different trustLevel scores

# Prerequisites

### Packages and Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn import svm as SVM
from sklearn.naive_bayes import GaussianNB as NB
from xgboost import XGBClassifier

In [2]:
# custom imports
from funcs import plot_cv_confidence_vs_profit, score_dmc_profit,dmc_profit,cv_preds_and_confusion_matrix,cv_profits_for_models, profit_scoring
from customClassifiers import CustomModelWithThreshold, TrustHard, PerceptronLearner
from pipes import CustomAttributeAdder,Scaling,RandomAttributeAdder,Transformer,ClfSwitcher

from sklearn.base import BaseEstimator
from sklearn.linear_model import SGDClassifier

from sklearn.ensemble import VotingClassifier

In [3]:
# use sklearn pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import itertools

### Evaluation Function 

In [4]:
# Nico's script
import pandas as pd
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer

cv = StratifiedKFold(n_splits=10, random_state=42)
def profit_scorer(y, y_pred):
    profit_matrix = {(0,0): 0, (0,1): -5, (1,0): -25, (1,1): 5}
    return sum(profit_matrix[(pred, actual)] for pred, actual in zip(y_pred, y))
profit_scoring = make_scorer(profit_scorer, greater_is_better=True)

### Load the Data

In [5]:
# load the data
df_train = pd.read_csv('train.csv' ,delimiter="|")
df_test = pd.read_csv('test.csv', delimiter="|")
X_train, y_train = df_train.drop(columns='fraud'), df_train['fraud']
#y_test = test.pop('fraud')

In [6]:
print(X_train.shape)
X_train.head(3)

(1879, 9)


,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769


# trustLevel==1

### Split the Data

In [59]:
# create X_train trustLevel==1
df_train = pd.read_csv('train.csv' ,delimiter="|")

is_trust1 = df_train['trustLevel']==1
df_train_trust1 = df_train[is_trust1]

X_train_trust1, y_train_trust1 = df_train_trust1.drop(columns='fraud'), df_train_trust1['fraud']

print(X_train_trust1.shape)
print(y_train_trust1.shape)
X_train_trust1.head(3)

(332, 9)
(332,)


,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
5,1,770,11.09,11,5,2,0.033766,0.014403,0.423077
15,1,870,32.45,3,1,5,0.006897,0.037299,0.500000
24,1,71,78.91,1,4,4,0.014085,1.111408,1.000000


In [60]:
# show distribution
from collections import Counter

print("training data size trustLevel==1: {}".format(len(y_train_trust1)))
print(sorted(Counter(y_train_trust1).items()))
print("Max. profit: {}".format(89*5))

training data size trustLevel==1: 332
[(0, 243), (1, 89)]
Max. profit: 445


**Note:** Maximal profit equals **445**

### Preprocess the Data

In [9]:
# select one or more out of feature list below that will be added in the featureGenerationPipeline
feature_list = ['scannedLineItemsTotal',
                'valuePerLineItem',
                'quantityModificationsPerLineItem',
                'lineItemVoids*scansWithoutRegistration',
                #'totalScanTimeInSeconds/trustLevel',
                #'trustLevel_Log', 
               ]

**Note:** Only two preprocessing steps at the moment are adding newly designed features (see above) and scaling

In [10]:
featureGeneration_pipeline = Pipeline([
    ("attribs_adder", CustomAttributeAdder(featurelist=feature_list)),                
    #("RandomAttributeAdder", RandomAttributeAdder())         #  This class is still void
    ])


preprocessing_pipeline = Pipeline([
    #("transformer", Transformer()),                           # This class is still void
    ("scaler", Scaling(strategy='Standard')),
])

In [11]:
# combine two pipeline into a single data_preparation_pipeline
data_preparation_pipeline = Pipeline([
    ('feature_generation', featureGeneration_pipeline),
    ('preprocessing', preprocessing_pipeline)
])

X_train_trust1_prepared = data_preparation_pipeline.fit_transform(X_train_trust1)

/home/roman/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/roman/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [12]:
# delete trustLevel feature!
X_train_trust1_prepared = np.delete(X_train_trust1_prepared, 0, 1)

In [13]:
X_train_trust1_prepared.shape

(332, 12)

### Model Selection

In [26]:
# Install the library for Bayesian optimization from here: https://github.com/fmfn/BayesianOptimization
from bayes_opt import BayesianOptimization

### Logistic Regression

In [15]:
def evaluateLogistic(C):
    
    model = LogisticRegression(C=C, solver='liblinear', random_state=42)
    
    return np.mean(cross_validate(model, X_train_trust1_prepared, y=y_train_trust1, cv=cv,
                                scoring=profit_scoring,
                                  #scoring="f1"
                                 )['test_score'])

In [16]:
params_logistic = {
    'C': (1,35),
}

In [ ]:
optimization_logistic = BayesianOptimization(evaluateLogistic, params_logistic, random_state=231)
optimization_logistic.maximize(n_iter=50, init_points=50)

In [ ]:
optimization_logistic.max

In [17]:
sum(cross_validate(LogisticRegression(C=2.32,
                                      solver='liblinear',
                                      random_state=42),
                   X_train_trust1_prepared,
                   y=y_train_trust1,
                   cv=cv,
                   scoring=profit_scoring)['test_score'])

335

In [18]:
335/445

0.7528089887640449

**Observation:** For **trustLevel==1** Logistic Regression achieves profit of **335** or **0.75** percent of maximal profit

### SGD

In [ ]:
def evaluateSgd(alpha, l1_ratio, tol, penalty, loss):
    
    # 3 options, l1 by default
    penalty_str = 'l1'
    if int(penalty) == 0:
        penalty_str = 'l2'
    elif int(penalty) == 1:
        penalty_str = 'elasticnet'
    
    # 3 options, modified_huber by default
    loss_str = 'modified_huber'
    if int(loss) == 0:
        loss_str = 'hinge'
    elif int(loss) == 1:
        loss_str = 'log'
        
    
    model = SGDClassifier(alpha=alpha, l1_ratio=l1_ratio, tol=tol,
                          penalty=penalty_str, loss=loss_str, random_state=231)
    
    # This integrates sampling into the training. Trains on oversampled data but evaluates on unsampled data
    #return cross_val_imbalanced(model, X_train_prepared, y_train, RandomOverSampler(random_state=42))
    
    # this trains the classifier on the unbalanced folds
    return sum(cross_validate(model, X_train_trust1_prepared, y=y_train_trust1, cv=cv,
                              #scoring="f1",
                              scoring=profit_scoring,
                             )['test_score'])

In [ ]:
params_sgd = {
    'alpha': (1e-6, 1),
    'l1_ratio': (0, 1),
    'tol': (1e-9, 1e-1),
    'penalty': (0, 3),
    'loss': (0, 3)
}

In [ ]:
optimization_sgd = BayesianOptimization(evaluateSgd, params_sgd, random_state=231)
optimization_sgd.maximize(n_iter=200, init_points=20)

In [ ]:
optimization_sgd.max

In [19]:
sum(cross_validate(SGDClassifier(alpha=0.448373,
                                 l1_ratio=0.0859590,
                                 tol=0.0403,
                                 penalty='elasticnet',
                                 loss='modified_huber',
                                 random_state=231),
                   X_train_trust1_prepared,
                   y=y_train_trust1,
                   cv=cv,
                   scoring=profit_scoring)['test_score'])

320

### BorderLineSMOTE + SGD

In [20]:
# import sampling classes
from imblearn.pipeline import make_pipeline

# oversampling
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SVMSMOTE

# combination of over and undersampling
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

In [21]:
def evaluateSgd(alpha,
                l1_ratio,
                tol,
                n_neighbors,
                #k_neighbors,
                #m_neighbors
               ):
    
    sampler = RandomOverSampler(random_state=231,
                     #n_neighbors=int(n_neighbors),
                     #sampling_strategy='all',
                     #k_neighbors=int(k_neighbors),
                     #m_neighbors=int(m_neighbors)
                    )
    
    model = SGDClassifier(alpha=alpha,
                          l1_ratio=l1_ratio,
                          tol=tol,
                          penalty='elasticnet',
                          loss='modified_huber',
                          random_state=231)
    
    sampler_model_pipeline = make_pipeline(sampler,
                                          model)
    
    # this trains the classifier on the unbalanced folds
    return sum(cross_validate(sampler_model_pipeline, X_train_trust1_prepared, y=y_train_trust1, cv=cv,
                              #scoring="f1",
                              scoring=profit_scoring,
                             )['test_score'])

In [ ]:
params_sgd = {
    'alpha': (1e-6, 1),
    'l1_ratio': (0, 1),
    'tol': (1e-9, 1e-1),
    'n_neighbors': (2, 20),
    #'k_neighbors': (2, 20),
    #'m_neighbors': (2, 20)
}

In [ ]:
optimization_sgd = BayesianOptimization(evaluateSgd, params_sgd, random_state=231)
optimization_sgd.maximize(n_iter=200, init_points=50)

In [ ]:
optimization_sgd.max

### AdaBoost + Logistic Regression 

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
def evaluateAda(C,
                n_estimators,
                learning_rate,
               ):
    
    
    model = AdaBoostClassifier(base_estimator=LogisticRegression(C=C,
                                                                 solver='liblinear',
                                                                 random_state=42),
                               n_estimators=int(n_estimators),
                               learning_rate=learning_rate,
                               random_state=231)
    
    # this trains the classifier on the unbalanced folds
    return sum(cross_validate(model, X_train_trust1_prepared, y=y_train_trust1, cv=cv,
                              #scoring="f1",
                              scoring=profit_scoring,
                             )['test_score'])

In [ ]:
params_ada = {
    'C': (1, 100),
    'n_estimators': (50, 500),
    'learning_rate': (0.01, 1),
}

In [ ]:
optimization_ada = BayesianOptimization(evaluateAda, params_ada, random_state=231)
optimization_ada.maximize(n_iter=200, init_points=20)

In [ ]:
optimization_ada.max

### XGB - booster:'gblinear'

In [ ]:
from xgboost import XGBClassifier

In [ ]:
def evaluateXGB(reg_alpha,
                reg_lambda,
                #subsample, colsample_bytree,
                #gamma,
                #max_depth,
                #min_child_weight,
                n_estimators,
               ):
    
    model = XGBClassifier(reg_lambda=reg_lambda,
                          reg_alpha=reg_alpha,
                          updater='coord_descent',
                          learning_rate =0.01,
                          n_estimators=int(n_estimators),
                          
                          #max_depth=int(max_depth),
                          #min_child_weight=int(min_child_weight),
                          #gamma=gamma,
                          #subsample=subsample,
                          #colsample_bytree=colsample_bytree,
                          #reg_alpha=reg_alpha,
                          #reg_lambda=reg_lambda,
                          
                          objective= 'binary:logistic',
                          booster='gblinear',
                          n_jobs=-1,
                          scale_pos_weight=1,
                          seed=231)
    
    return sum(cross_validate(model, X_train_trust1_prepared, y=y_train_trust1, cv=cv,
                              scoring=profit_scoring,
                              #scoring="f1",
                              #scoring="precision"
                              n_jobs=-1)['test_score'])

In [ ]:
params_XGB = {
    'n_estimators': (50,1000),
    #'max_depth':(3,10.5),
    #'min_child_weight':(1,6.5),
    #'gamma': (0,0.6),
    #'subsample':(0.6,1),
    #'colsample_bytree':(0.6,1),
    'reg_alpha':(0, 100),
    'reg_lambda':(0, 100)
}

In [ ]:
optimization_XGB = BayesianOptimization(evaluateXGB, params_XGB, random_state=42)
optimization_XGB.maximize(n_iter=200, init_points=50)

In [ ]:
optimization_XGB.max

### XGB - booster:'gbtree'

In [ ]:
from xgboost import XGBClassifier

In [ ]:
def evaluateXGB(reg_alpha,
                #reg_lambda,
                subsample, colsample_bytree,
                gamma,
                max_depth,
                min_child_weight,
                n_estimators,
               ):
    
    model = XGBClassifier(learning_rate =0.01,
                          n_estimators=int(n_estimators),
                          max_depth=int(max_depth),
                          min_child_weight=int(min_child_weight),
                          gamma=gamma,
                          subsample=subsample,
                          colsample_bytree=colsample_bytree,
                          reg_alpha=reg_alpha,
                          #reg_lambda=reg_lambda,
                          objective= 'binary:logistic',
                          booster='gbtree',
                          n_jobs=-1,
                          scale_pos_weight=1,
                          seed=231)
    
    return sum(cross_validate(model, X_train_trust1_prepared, y=y_train_trust1, cv=cv,
                              scoring=profit_scoring,
                              #scoring="f1",
                              #scoring="precision"
                              n_jobs=-1)['test_score'])

In [ ]:
params_XGB = {
    'n_estimators': (1,1000),
    'max_depth':(3,10.5),
    'min_child_weight':(1,6.5),
    'gamma': (0,0.6),
    'subsample':(0.6,1),
    'colsample_bytree':(0.6,1),
    'reg_alpha':(0.0005, 100),
    #'reg_lambda':(0.3, 0.7)
}

In [ ]:
optimization_XGB = BayesianOptimization(evaluateXGB, params_XGB, random_state=42)
optimization_XGB.maximize(n_iter=200, init_points=50)

In [ ]:
optimization_XGB.max

# trustLevel==2

### Split the Data

In [22]:
# create X_train trustLevel==2
df_train = pd.read_csv('train.csv' ,delimiter="|")

is_trust2 = df_train['trustLevel']==2
df_train_trust2 = df_train[is_trust2]

X_train_trust2, y_train_trust2 = df_train_trust2.drop(columns='fraud'), df_train_trust2['fraud']

print(X_train_trust2.shape)
print(y_train_trust2.shape)
X_train_trust2.head(3)

(347, 9)
(347,)


,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
7,2,1545,22.80,0,8,4,0.006472,0.014757,0.000000
9,2,725,41.08,10,2,4,0.037241,0.056662,0.370370
23,2,125,25.50,5,6,2,0.192000,0.204000,0.208333


In [23]:
# show distribution
from collections import Counter

print("training data size trustLevel==1: {}".format(len(y_train_trust2)))
print(sorted(Counter(y_train_trust2).items()))
print("Max. profit: {}".format(15*5))

training data size trustLevel==1: 347
[(0, 332), (1, 15)]
Max. profit: 75


**Note:** Maximal profit equals **75**

### Preprocess the Data

In [24]:
# select one or more out of feature list below that will be added in the featureGenerationPipeline
feature_list = ['scannedLineItemsTotal',
                'valuePerLineItem',
                'quantityModificationsPerLineItem',
                'lineItemVoids*scansWithoutRegistration',
                #'totalScanTimeInSeconds/trustLevel',
                #'trustLevel_Log', 
               ]

**Note:** Only two preprocessing steps at the moment are adding newly designed features (see above) and scaling

In [25]:
featureGeneration_pipeline = Pipeline([
    ("attribs_adder", CustomAttributeAdder(featurelist=feature_list)),                
    #("RandomAttributeAdder", RandomAttributeAdder())         #  This class is still void
    ])


preprocessing_pipeline = Pipeline([
    #("transformer", Transformer()),                           # This class is still void
    ("scaler", Scaling(strategy='Standard')),
])

In [26]:
# combine two pipeline into a single data_preparation_pipeline
data_preparation_pipeline = Pipeline([
    ('feature_generation', featureGeneration_pipeline),
    ('preprocessing', preprocessing_pipeline)
])

X_train_trust2_prepared = data_preparation_pipeline.fit_transform(X_train_trust2)

/home/roman/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/roman/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [27]:
# delete trust level
X_train_trust2_prepared = np.delete(X_train_trust2_prepared, 0, 1)

In [28]:
X_train_trust2_prepared.shape

(347, 12)

### Model Selection

### Logistic Regression

In [ ]:
def evaluateLogistic(C):
    
    model = LogisticRegression(C=C, solver='liblinear', random_state=42)
    
    return np.mean(cross_validate(model, X_train_trust2_prepared, y=y_train_trust2, cv=cv,
                              #scoring=profit_scoring,
                             scoring="precision",
                             )['test_score'])

In [ ]:
params_logistic = {
    'C': (1,1000),
}

In [ ]:
optimization_logistic = BayesianOptimization(evaluateLogistic, params_logistic, random_state=231)
optimization_logistic.maximize(n_iter=50, init_points=50)

In [ ]:
optimization_logistic.max

In [29]:
sum(cross_validate(LogisticRegression(C=140.91,
                                      solver='liblinear',
                                      random_state=42),
                   X_train_trust2_prepared,
                   y=y_train_trust2,
                   cv=cv,
                   scoring=profit_scoring)['test_score'])

-55

**Observation:** For **trustLevel==2** Logistic Regression achieves profit of **-55**

### SGD

In [ ]:
def evaluateSgd(alpha, l1_ratio, tol, penalty, loss):
    
    # 3 options, l1 by default
    penalty_str = 'l1'
    if int(penalty) == 0:
        penalty_str = 'l2'
    elif int(penalty) == 1:
        penalty_str = 'elasticnet'
    
    # 3 options, modified_huber by default
    loss_str = 'modified_huber'
    if int(loss) == 0:
        loss_str = 'hinge'
    elif int(loss) == 1:
        loss_str = 'log'
        
    
    model = SGDClassifier(alpha=alpha, l1_ratio=l1_ratio, tol=tol,
                          penalty=penalty_str, loss=loss_str, random_state=231)
    
    # This integrates sampling into the training. Trains on oversampled data but evaluates on unsampled data
    #return cross_val_imbalanced(model, X_train_prepared, y_train, RandomOverSampler(random_state=42))
    
    # this trains the classifier on the unbalanced folds
    return sum(cross_validate(model, X_train_trust2_prepared, y=y_train_trust2, cv=cv,
                              scoring=profit_scoring)['test_score'])

In [ ]:
params_sgd = {
    'alpha': (1e-6, 1),
    'l1_ratio': (0, 1),
    'tol': (1e-9, 1e-1),
    'penalty': (0, 3),
    'loss': (0, 3)
}

In [ ]:
optimization_sgd = BayesianOptimization(evaluateSgd, params_sgd, random_state=231)
optimization_sgd.maximize(n_iter=200, init_points=20)

In [ ]:
optimization_sgd.max

In [30]:
sum(cross_validate(SGDClassifier(alpha=0.002964,
                                 l1_ratio=0.702337,
                                 tol=0.022649,
                                 penalty='elasticnet',
                                 loss='modified_huber',
                                 random_state=231),
                   X_train_trust2_prepared,
                   y=y_train_trust2,
                   cv=cv,
                   scoring=profit_scoring)['test_score'])

10

### XGB

In [31]:
from xgboost import XGBClassifier

In [ ]:
def evaluateXGB(reg_alpha,
                #reg_lambda,
                subsample, colsample_bytree,
                gamma,
                max_depth,
                #min_child_weight,
                n_estimators,
               ):
    
    model = XGBClassifier(learning_rate =0.01,
                          n_estimators=int(n_estimators),
                          max_depth=int(max_depth),
                          min_child_weight=1,
                          gamma=gamma,
                          subsample=subsample,
                          colsample_bytree=colsample_bytree,
                          reg_alpha=reg_alpha,
                          #reg_lambda=reg_lambda,
                          objective= 'binary:logistic',
                          n_jobs=-1,
                          scale_pos_weight=1,
                          seed=231)
    
    return sum(cross_validate(model, X_train_trust2_prepared, y=y_train_trust2, cv=cv,
                              scoring=profit_scoring,
                              #scoring="f1",
                              #scoring="precision"
                              n_jobs=-1)['test_score'])

In [ ]:
params_XGB = {
    'n_estimators': (500,2500),
    'max_depth':(7.5,9.5),
    #'min_child_weight':(1,7),
    'gamma': (0,0.1),
    'subsample':(0.85,1),
    'colsample_bytree':(0.85,1),
    'reg_alpha':(0.001, 1),
    #'reg_lambda':(0.3, 0.7)
}

In [ ]:
optimization_XGB = BayesianOptimization(evaluateXGB, params_XGB, random_state=42)
optimization_XGB.maximize(n_iter=200, init_points=50)

In [ ]:
optimization_XGB.max

In [32]:
sum(cross_validate(XGBClassifier(learning_rate =0.1,
                          n_estimators=331,
                          max_depth=8,
                          min_child_weight=1,
                          gamma=0.032443,
                          subsample=0.905887,
                          colsample_bytree=0.93342,
                          reg_alpha=0.782748,
                          #reg_lambda=0.5123,
                          objective= 'binary:logistic',
                          n_jobs=-1,
                          scale_pos_weight=1,
                          seed=231),
                   X_train_trust2_prepared,
                   y=y_train_trust2,
                   cv=cv,
                   scoring=profit_scoring)['test_score'])

25

# trustLevel1 & 2

### Split the Data

In [33]:
# create X_train trustLevel==1
df_train = pd.read_csv('train.csv' ,delimiter="|")

is_trust12 = df_train['trustLevel']<=2
df_train_trust12 = df_train[is_trust12]

X_train_trust12, y_train_trust12 = df_train_trust12.drop(columns='fraud'), df_train_trust12['fraud']

print(X_train_trust12.shape)
print(y_train_trust12.shape)
X_train_trust12.head(3)

(679, 9)
(679,)


,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
5,1,770,11.09,11,5,2,0.033766,0.014403,0.423077
7,2,1545,22.80,0,8,4,0.006472,0.014757,0.000000
9,2,725,41.08,10,2,4,0.037241,0.056662,0.370370


In [34]:
# show distribution
from collections import Counter

print("training data size trustLevel==1&2 combined: {}".format(len(y_train_trust12)))
print(sorted(Counter(y_train_trust12).items()))
print("Max. Profit: {}".format(104*5))

training data size trustLevel==1&2 combined: 679
[(0, 575), (1, 104)]
Max. Profit: 520


**Note:** Maximal profit equals **520**

In [35]:
# keep trustLevel as a feature but recode it to be binary [0,1]
X_train_trust12['trustLevel'].replace(to_replace=1, value=0, inplace=True)
X_train_trust12['trustLevel'].replace(to_replace=2, value=1, inplace=True)

In [36]:
X_train_trust12.head(3)

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
5,0,770,11.09,11,5,2,0.033766,0.014403,0.423077
7,1,1545,22.80,0,8,4,0.006472,0.014757,0.000000
9,1,725,41.08,10,2,4,0.037241,0.056662,0.370370


### Preprocess the Data

In [37]:
# select one or more out of feature list below that will be added in the featureGenerationPipeline
feature_list = ['scannedLineItemsTotal',
                'valuePerLineItem',
                'quantityModificationsPerLineItem',
                'lineItemVoids*scansWithoutRegistration',
                #'totalScanTimeInSeconds/trustLevel',
                #'trustLevel_Log', 
               ]

**Note:** Only two preprocessing steps at the moment are adding newly designed features (see above) and scaling

In [38]:
featureGeneration_pipeline = Pipeline([
    ("attribs_adder", CustomAttributeAdder(featurelist=feature_list)),                
    #("RandomAttributeAdder", RandomAttributeAdder())         #  This class is still void
    ])

**Note:** This is a slight change to the preprocesses pipeline to treat trustLevel as a categorical feature and do not apply scaling to it (I'm pretty sure there is a smarter way of achieving this)

In [39]:
# define a class: that will take pandas dataframe select columns and convert to numpy array
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [40]:
X_train_trust12_num = X_train_trust12.drop(columns='trustLevel')
X_train_trust12_num.columns

Index(['totalScanTimeInSeconds', 'grandTotal', 'lineItemVoids',
       'scansWithoutRegistration', 'quantityModifications',
       'scannedLineItemsPerSecond', 'valuePerSecond',
       'lineItemVoidsPerPosition'],
      dtype='object')

In [41]:
# use DataFrameSelector
num_attributes = list(X_train_trust12_num)
num_attributes.append('scannedLineItemsTotal')
num_attributes.append('valuePerLineItem')
num_attributes.append('quantityModificationsPerLineItem')
num_attributes.append('lineItemVoids*scansWithoutRegistration')
cat_attributes = ["trustLevel"]

In [42]:
num_attributes

['totalScanTimeInSeconds',
 'grandTotal',
 'lineItemVoids',
 'scansWithoutRegistration',
 'quantityModifications',
 'scannedLineItemsPerSecond',
 'valuePerSecond',
 'lineItemVoidsPerPosition',
 'scannedLineItemsTotal',
 'valuePerLineItem',
 'quantityModificationsPerLineItem',
 'lineItemVoids*scansWithoutRegistration']

In [43]:
num_pipeline = Pipeline([
    ("selector", DataFrameSelector(num_attributes)),
    ("scaler", Scaling(strategy='Standard')),
    ])

cat_pipeline = Pipeline([
    ("selector", DataFrameSelector(cat_attributes)),
    ])


In [44]:
# combining numeric and categorical preprocessing with FeatureUnion
from sklearn.pipeline import FeatureUnion

preprocessing_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

In [45]:
# combine two pipeline into a single data_preparation_pipeline
data_preparation_pipeline = Pipeline([
    ('feature_generation', featureGeneration_pipeline),
    ('preprocessing', preprocessing_pipeline)
])

X_train_trust12_prepared = data_preparation_pipeline.fit_transform(X_train_trust12)

In [46]:
X_train_trust12_prepared.shape

(679, 13)

In [47]:
X_train_trust12_prepared

array([[-0.26493495, -1.27906871,  1.59381746, ..., -0.42462227,
         1.09821241,  0.        ],
       [ 1.18451551, -0.8789435 , -1.61919808, ...,  0.05885284,
        -1.02185699,  1.        ],
       [-0.34909659, -0.25432447,  1.30172513, ..., -0.31803604,
        -0.25092266,  1.        ],
       ...,
       [-1.10468108,  0.93990064,  0.71754049, ..., -0.40960751,
         1.13675913,  0.        ],
       [-0.96254142, -0.22664715, -0.15873648, ..., -0.53973539,
        -0.05818908,  0.        ],
       [-0.42390693,  0.4837374 , -1.32710576, ..., -0.41502951,
        -0.79057669,  1.        ]])

## XGB

In [30]:
from xgboost import XGBClassifier

In [31]:
def evaluateXGB(reg_alpha,
                #reg_lambda,
                subsample, colsample_bytree,
                gamma,
                max_depth,
                min_child_weight,
                n_estimators,
               ):
    
    model = XGBClassifier(learning_rate =0.1,
                          n_estimators=int(n_estimators),
                          max_depth=int(max_depth),
                          min_child_weight=(min_child_weight),
                          gamma=gamma,
                          subsample=subsample,
                          colsample_bytree=colsample_bytree,
                          reg_alpha=reg_alpha,
                          #reg_lambda=reg_lambda,
                          objective= 'binary:logistic',
                          n_jobs=-1,
                          scale_pos_weight=1,
                          seed=231)
    
    return sum(cross_validate(model, X_train_trust12_prepared, y=y_train_trust12, cv=cv,
                              scoring=profit_scoring,
                              #scoring="f1",
                              #scoring="precision"
                              n_jobs=-1)['test_score'])

In [32]:
params_XGB = {
    'n_estimators': (50,500),
    'max_depth':(3,10.5),
    'min_child_weight':(1,7),
    'gamma': (0,1),
    'subsample':(0.6,1),
    'colsample_bytree':(0.6,1),
    'reg_alpha':(0.001, 100),
    #'reg_lambda':(0.3, 0.7)
}

In [33]:
optimization_XGB = BayesianOptimization(evaluateXGB, params_XGB, random_state=42)
optimization_XGB.maximize(n_iter=200, init_points=50)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | n_esti... | reg_alpha | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        | -55.0     |  0.7498   |  0.9507   |  8.49     |  4.592    |  120.2    |  15.6     |  0.6232   |
|  2        | -520.0    |  0.9465   |  0.6011   |  8.311    |  1.124    |  486.5    |  83.24    |  0.6849   |
|  3        | -430.0    |  0.6727   |  0.1834   |  5.282    |  4.149    |  244.4    |  29.12    |  0.8447   |
|  4        | -45.0     |  0.6558   |  0.2921   |  5.748    |  3.736    |  403.3    |  19.97    |  0.8057   |
|  5        | -520.0    |  0.837    |  0.04645  |  7.557    |  2.023    |  79.27    |  94.89    |  0.9863   |
|  6        |  5.0      |  0.9234   |  0.3046   |  3.733    |  5.105    |  248.1    |  12.2     |  0.7981   |
|  7        | -520.0    |  0.6138   |  0.9093   |  4.941    |  4.975    |  190.3    |  52.01    |  0.8187   |
|  8      

|  45       | -520.0    |  0.9181   |  0.2708   |  6.292    |  1.471    |  61.41    |  96.26    |  0.9344   |
|  46       | -520.0    |  0.8784   |  0.409    |  4.3      |  1.939    |  162.6    |  54.92    |  0.8858   |
|  47       | -520.0    |  0.8641   |  0.2799   |  10.16    |  5.427    |  299.5    |  61.17    |  0.7678   |
|  48       |  40.0     |  0.6991   |  0.356    |  8.684    |  1.086    |  102.2    |  4.601    |  0.6163   |
|  49       | -520.0    |  0.9422   |  0.7037   |  6.556    |  1.587    |  271.2    |  47.35    |  0.6693   |
|  50       | -520.0    |  0.7735   |  0.3985   |  7.619    |  4.811    |  70.39    |  37.46    |  0.8503   |
|  51       |  140.0    |  0.94     |  0.3399   |  5.868    |  1.482    |  311.5    |  3.662    |  0.7781   |
|  52       |  140.0    |  0.9864   |  0.3304   |  5.678    |  1.675    |  308.0    |  3.413    |  0.8018   |
|  53       |  130.0    |  0.9533   |  0.3646   |  6.115    |  1.865    |  309.7    |  4.068    |  0.7838   |
|  54     

|  90       |  220.0    |  0.6059   |  0.3272   |  4.661    |  1.751    |  309.9    |  1.691    |  0.725    |
|  91       |  125.0    |  0.8519   |  0.868    |  5.43     |  2.008    |  311.1    |  2.506    |  0.8394   |
|  92       |  130.0    |  0.6112   |  0.1496   |  7.16     |  2.314    |  177.8    |  3.797    |  0.8792   |
|  93       |  90.0     |  0.6975   |  0.2712   |  6.944    |  1.835    |  410.4    |  7.285    |  0.9336   |
|  94       |  130.0    |  0.9319   |  0.08713  |  6.282    |  1.712    |  309.9    |  4.19     |  0.6807   |
|  95       | -30.0     |  0.8335   |  0.9356   |  7.779    |  6.714    |  411.5    |  18.79    |  0.9511   |
|  96       |  70.0     |  0.7093   |  0.338    |  6.084    |  2.088    |  308.0    |  4.435    |  0.7411   |
|  97       |  130.0    |  0.8635   |  0.2412   |  5.088    |  1.238    |  308.0    |  3.509    |  0.674    |
|  98       |  185.0    |  0.8809   |  0.4064   |  5.09     |  1.87     |  307.9    |  3.043    |  0.8532   |
|  99     

|  135      |  145.0    |  0.7764   |  0.3223   |  6.079    |  1.907    |  310.5    |  3.051    |  0.7077   |
|  136      |  185.0    |  0.7603   |  0.5984   |  5.047    |  2.824    |  308.0    |  2.715    |  0.6808   |
|  137      |  160.0    |  0.987    |  0.167    |  6.088    |  2.699    |  308.4    |  3.897    |  0.6173   |
|  138      |  105.0    |  0.7919   |  0.7134   |  4.591    |  1.247    |  307.6    |  3.775    |  0.7924   |
|  139      |  100.0    |  0.8339   |  0.5315   |  6.04     |  1.239    |  311.2    |  3.935    |  0.7476   |
|  140      |  90.0     |  0.6314   |  0.2532   |  6.746    |  1.454    |  410.9    |  7.513    |  0.9573   |
|  141      |  40.0     |  0.8412   |  0.6673   |  3.964    |  5.093    |  247.2    |  12.06    |  0.9585   |
|  142      |  0.0      |  0.8888   |  0.9745   |  3.412    |  4.543    |  247.6    |  12.83    |  0.847    |
|  143      |  185.0    |  0.953    |  0.3401   |  5.722    |  1.756    |  310.8    |  2.204    |  0.7383   |
|  144    

|  181      |  195.0    |  0.981    |  0.6171   |  3.425    |  1.313    |  348.5    |  0.2262   |  0.6899   |
|  182      |  120.0    |  0.784    |  0.1646   |  4.352    |  2.026    |  308.2    |  3.033    |  0.853    |
|  183      |  160.0    |  0.9453   |  0.1447   |  5.113    |  2.174    |  308.3    |  2.996    |  0.7827   |
|  184      |  85.0     |  0.6544   |  0.4095   |  5.977    |  2.512    |  309.7    |  4.166    |  0.9882   |
|  185      |  45.0     |  0.7856   |  0.3149   |  8.884    |  2.497    |  330.5    |  9.278    |  0.89     |
|  186      |  120.0    |  0.9978   |  0.8345   |  4.672    |  2.575    |  307.9    |  2.441    |  0.8585   |
|  187      | -65.0     |  0.8668   |  0.5888   |  3.481    |  4.575    |  247.8    |  13.25    |  0.664    |
|  188      |  65.0     |  0.6208   |  0.4166   |  5.977    |  1.658    |  307.5    |  4.397    |  0.731    |
|  189      |  60.0     |  0.9242   |  0.2185   |  4.08     |  5.15     |  248.4    |  11.53    |  0.9893   |
|  190    

|  227      | -90.0     |  0.9913   |  0.4703   |  4.395    |  4.816    |  247.9    |  13.07    |  0.6961   |
|  228      |  155.0    |  0.8123   |  0.07254  |  5.56     |  1.427    |  310.1    |  2.978    |  0.684    |
|  229      |  110.0    |  0.7001   |  0.6939   |  4.884    |  1.526    |  311.5    |  2.748    |  0.7694   |
|  230      |  155.0    |  0.8152   |  0.4726   |  5.098    |  1.099    |  309.5    |  2.811    |  0.6634   |
|  231      |  155.0    |  0.7556   |  0.6531   |  5.612    |  2.004    |  311.8    |  2.199    |  0.8569   |
|  232      |  100.0    |  0.6231   |  0.2455   |  5.184    |  1.419    |  308.3    |  3.638    |  0.6784   |
|  233      |  115.0    |  0.9122   |  0.3509   |  6.879    |  3.74     |  178.6    |  3.87     |  0.766    |
|  234      |  125.0    |  0.7712   |  0.3362   |  5.55     |  1.362    |  307.6    |  4.116    |  0.9379   |
|  235      |  45.0     |  0.6855   |  0.8238   |  8.289    |  2.516    |  329.4    |  8.25     |  0.9372   |
|  236    

In [34]:
optimization_XGB.max

{'target': 220.0,
 'params': {'colsample_bytree': 0.605873136934962,
  'gamma': 0.327183911209176,
  'max_depth': 4.660877147619236,
  'min_child_weight': 1.7514049980791158,
  'n_estimators': 309.86651749531757,
  'reg_alpha': 1.691261657814916,
  'subsample': 0.7249958039005243}}

In [50]:
sum(cross_validate(XGBClassifier(learning_rate =0.1,
                          n_estimators=308,
                          max_depth=4,
                          min_child_weight=1,
                          gamma=0.327183911209176,
                          subsample=0.7249958039005243,
                          colsample_bytree=0.605873136934962,
                          reg_alpha=1.691261657814916,
                          #reg_lambda=0.5123,
                          objective= 'binary:logistic',
                          n_jobs=-1,
                          scale_pos_weight=1,
                          seed=231),
                   X_train_trust12_prepared,
                   y=y_train_trust12,
                   cv=cv,
                   scoring=profit_scoring)['test_score'])

155

### Logistic Regression

In [51]:
def evaluateLogistic(C):
    
    model = LogisticRegression(C=C, solver='liblinear', random_state=42)
    
    return sum(cross_validate(model, X_train_trust12_prepared, y=y_train_trust12, cv=cv,
                             scoring=profit_scoring,
                             #scoring="precision",
                             )['test_score'])

In [52]:
params_logistic = {
    'C': (1,1000),
}

In [53]:
optimization_logistic = BayesianOptimization(evaluateLogistic, params_logistic, random_state=231)
optimization_logistic.maximize(n_iter=100, init_points=50)

|   iter    |  target   |     C     |
-------------------------------------
|  1        |  260.0    |  781.7    |
|  2        |  270.0    |  584.6    |
|  3        |  270.0    |  427.3    |
|  4        |  270.0    |  316.0    |
|  5        |  260.0    |  827.1    |
|  6        |  260.0    |  903.5    |
|  7        |  330.0    |  39.9     |
|  8        |  260.0    |  915.3    |
|  9        |  330.0    |  61.44    |
|  10       |  280.0    |  181.5    |
|  11       |  330.0    |  73.23    |
|  12       |  270.0    |  459.3    |
|  13       |  260.0    |  708.7    |
|  14       |  270.0    |  483.8    |
|  15       |  330.0    |  26.42    |
|  16       |  330.0    |  67.84    |
|  17       |  270.0    |  426.5    |
|  18       |  280.0    |  220.0    |
|  19       |  260.0    |  810.3    |
|  20       |  280.0    |  140.9    |
|  21       |  260.0    |  751.4    |
|  22       |  245.0    |  279.8    |
|  23       |  260.0    |  949.2    |
|  24       |  260.0    |  885.1    |
|  25       

|  132      |  330.0    |  66.74    |
|  133      |  330.0    |  72.36    |
|  134      |  330.0    |  37.58    |
|  135      |  330.0    |  41.57    |
|  136      |  330.0    |  64.09    |
|  137      |  330.0    |  25.46    |
|  138      |  330.0    |  24.87    |
|  139      |  330.0    |  25.64    |
|  140      |  330.0    |  24.14    |
|  141      |  330.0    |  24.82    |
|  142      |  330.0    |  23.38    |
|  143      |  330.0    |  24.07    |
|  144      |  330.0    |  22.56    |
|  145      |  330.0    |  23.23    |
|  146      |  330.0    |  21.69    |
|  147      |  330.0    |  22.45    |
|  148      |  330.0    |  20.78    |
|  149      |  330.0    |  21.51    |
|  150      |  330.0    |  19.81    |


In [54]:
optimization_logistic.max

{'target': 330.0, 'params': {'C': 39.89916730939222}}

In [56]:
sum(cross_validate(LogisticRegression(C=39.89916730939222,
                                      solver='liblinear',
                                      random_state=42),
                   X_train_trust12_prepared,
                   y=y_train_trust12,
                   cv=cv,
                   scoring=profit_scoring)['test_score'])

330

# Add additional Features - Interaction Effects

# PolynomialFeatures - SelectKBest for trustLevel==1 

### Split the Data

In [21]:
# create X_train trustLevel==1
df_train = pd.read_csv('train.csv' ,delimiter="|")

is_trust1 = df_train['trustLevel']==1
df_train_trust1 = df_train[is_trust1]

X_train_trust1, y_train_trust1 = df_train_trust1.drop(columns='fraud'), df_train_trust1['fraud']

print(X_train_trust1.shape)
print(y_train_trust1.shape)
X_train_trust1.head(3)

(332, 9)
(332,)


,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
5,1,770,11.09,11,5,2,0.033766,0.014403,0.423077
15,1,870,32.45,3,1,5,0.006897,0.037299,0.500000
24,1,71,78.91,1,4,4,0.014085,1.111408,1.000000


In [22]:
X_train_trust1.head(3)
X_train_trust1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 332 entries, 5 to 1875
Data columns (total 9 columns):
trustLevel                   332 non-null int64
totalScanTimeInSeconds       332 non-null int64
grandTotal                   332 non-null float64
lineItemVoids                332 non-null int64
scansWithoutRegistration     332 non-null int64
quantityModifications        332 non-null int64
scannedLineItemsPerSecond    332 non-null float64
valuePerSecond               332 non-null float64
lineItemVoidsPerPosition     332 non-null float64
dtypes: float64(4), int64(5)
memory usage: 25.9 KB


In [23]:
# show distribution
from collections import Counter

print("training data size trustLevel==1: {}".format(len(y_train_trust1)))
print(sorted(Counter(y_train_trust1).items()))
print("Max. profit: {}".format(89*5))

training data size trustLevel==1: 332
[(0, 243), (1, 89)]
Max. profit: 445


**Note:** Maximal profit equals **445**

### Preprocess the Data

In [24]:
# drop trustLevel
X_train_trust1 = X_train_trust1.drop(columns='trustLevel')

**Note:** Only two preprocessing steps at the moment are adding newly designed features (see above) and scaling

In [25]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import mutual_info_classif

In [26]:
preprocessing_pipeline = Pipeline([
    #("transformer", Transformer()),                           # This class is still void
    #("interactions", PolynomialFeatures(2, interaction_only=True)),
    ("interactions", PolynomialFeatures(2, interaction_only=False)),
    ("scaler1", StandardScaler()),
    ("scaler", MinMaxScaler()),
    ("featureSelection", SelectKBest(mutual_info_classif, k=8)),
])

In [27]:
# apply new pipeline
X_train_trust1_prepared = preprocessing_pipeline.fit_transform(X_train_trust1, y=y_train_trust1)

In [28]:
# delete trustLevel feature!
#X_train_trust1_prepared = np.delete(X_train_trust1_prepared, 0, 1)

In [29]:
X_train_trust1_prepared.shape

(332, 6)

In [30]:
X_train_trust1_prepared

array([[2.06232423e-02, 4.23076923e-02, 8.62068966e-01, 4.65384615e-02,
        4.39135492e-04, 7.15398485e-05],
       [3.93980009e-03, 5.00000000e-02, 1.72413793e-01, 1.50000000e-02,
        1.82066175e-05, 2.18954215e-04],
       [8.40281669e-03, 1.00000000e-01, 1.11022302e-16, 1.00000000e-02,
        7.63073803e-05, 1.30485289e-02],
       ...,
       [1.30031476e-03, 2.33333333e-01, 6.89655172e-02, 1.63333333e-01,
        2.57920517e-06, 3.50525002e-04],
       [4.41460295e-02, 3.47826087e-02, 7.58620690e-01, 2.78260870e-02,
        1.97773894e-03, 9.67230464e-04],
       [4.03213306e-02, 1.92307692e-02, 8.62068966e-01, 9.61538462e-03,
        1.65228125e-03, 2.38234343e-04]])

### Model Selection

In [31]:
# Install the library for Bayesian optimization from here: https://github.com/fmfn/BayesianOptimization
from bayes_opt import BayesianOptimization

### Logistic Regression

In [32]:
def evaluateLogistic(C):
    
    model = LogisticRegression(C=C, solver='liblinear', random_state=42)
    
    return sum(cross_validate(model, X_train_trust1_prepared, y=y_train_trust1, cv=cv,
                                scoring=profit_scoring,
                                  #scoring="f1"
                                 )['test_score'])

In [33]:
params_logistic = {
    'C': (1,1000),
}

In [34]:
optimization_logistic = BayesianOptimization(evaluateLogistic, params_logistic, random_state=231)
optimization_logistic.maximize(n_iter=100, init_points=50)

|   iter    |  target   |     C     |
-------------------------------------
|  1        |  60.0     |  781.7    |
|  2        |  60.0     |  584.6    |
|  3        |  85.0     |  427.3    |
|  4        |  85.0     |  316.0    |
|  5        |  60.0     |  827.1    |
|  6        |  60.0     |  903.5    |
|  7        | -40.0     |  39.9     |
|  8        |  60.0     |  915.3    |
|  9        | -15.0     |  61.44    |
|  10       |  70.0     |  181.5    |
|  11       | -15.0     |  73.23    |
|  12       |  60.0     |  459.3    |
|  13       |  60.0     |  708.7    |
|  14       |  60.0     |  483.8    |
|  15       | -40.0     |  26.42    |
|  16       | -15.0     |  67.84    |
|  17       |  85.0     |  426.5    |
|  18       |  70.0     |  220.0    |
|  19       |  60.0     |  810.3    |
|  20       |  70.0     |  140.9    |
|  21       |  60.0     |  751.4    |
|  22       |  95.0     |  279.8    |
|  23       |  60.0     |  949.2    |
|  24       |  60.0     |  885.1    |
|  25       

KeyboardInterrupt: 

In [ ]:
optimization_logistic.max

In [17]:
sum(cross_validate(LogisticRegression(C=2.32,
                                      solver='liblinear',
                                      random_state=42),
                   X_train_trust1_prepared,
                   y=y_train_trust1,
                   cv=cv,
                   scoring=profit_scoring)['test_score'])

335

In [18]:
335/445

0.7528089887640449

**Observation:** For **trustLevel==1** Logistic Regression achieves profit of **335** or **0.75** percent of maximal profit

# SELECT FROM MODEL for trustLevel==1

### Split the Data

In [168]:
# create X_train trustLevel==1
df_train = pd.read_csv('train.csv' ,delimiter="|")

is_trust1 = df_train['trustLevel']==1
df_train_trust1 = df_train[is_trust1]

X_train_trust1, y_train_trust1 = df_train_trust1.drop(columns='fraud'), df_train_trust1['fraud']

print(X_train_trust1.shape)
print(y_train_trust1.shape)
X_train_trust1.head(3)

(332, 9)
(332,)


,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
5,1,770,11.09,11,5,2,0.033766,0.014403,0.423077
15,1,870,32.45,3,1,5,0.006897,0.037299,0.500000
24,1,71,78.91,1,4,4,0.014085,1.111408,1.000000


In [169]:
# show distribution
from collections import Counter

print("training data size trustLevel==1: {}".format(len(y_train_trust1)))
print(sorted(Counter(y_train_trust1).items()))
print("Max. profit: {}".format(89*5))

training data size trustLevel==1: 332
[(0, 243), (1, 89)]
Max. profit: 445


**Note:** Maximal profit equals **445**

In [170]:
# drop trustLevel
X_train_trust1 = X_train_trust1.drop(columns='trustLevel')

### Preprocess the Data

List of features that show at least some correlation with fraud according to Niklas' and Yaxi's notebooks:
- scannedLineItemsTotal * totalScanTimeInSeconds
- scannedLineItemsTotal * scansWithoutRegistration
- lineItemVoids * scannedLineItemsTotal
- grandTotal * scannedLineItemsTotal

In [171]:
# select one or more out of feature list below that will be added in the featureGenerationPipeline
feature_list = ['scannedLineItemsTotal',
                'valuePerLineItem',
                'quantityModificationsPerLineItem',
                'lineItemVoids*scansWithoutRegistration',
                #'totalScanTimeInSeconds/trustLevel',
                #'trustLevel_Log', 
               ]

**Note:** Only two preprocessing steps at the moment are adding newly designed features (see above) and scaling

In [172]:
featureGeneration_pipeline = Pipeline([
    ("attribs_adder", CustomAttributeAdder(featurelist=feature_list)),
    ("interactions", PolynomialFeatures(3, interaction_only=False))
    ])


preprocessing_pipeline = Pipeline([
    ("scaler", Scaling(strategy='Standard')),
])

In [173]:
# combine two pipeline into a single data_preparation_pipeline
data_preparation_pipeline = Pipeline([
    ('feature_generation', featureGeneration_pipeline),
    ('preprocessing', preprocessing_pipeline)
])

X_train_trust1_prepared = data_preparation_pipeline.fit_transform(X_train_trust1)

In [174]:
X_train_trust1_prepared.shape

(332, 455)

### Model Selection

In [175]:
# Install the library for Bayesian optimization from here: https://github.com/fmfn/BayesianOptimization
from bayes_opt import BayesianOptimization

### SelectFromModel + Logistic Regression

In [176]:
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

In [177]:
# generate fbeta scorer with beta = 0.5 giving more weight to precision as an alternative to cost evaluation
from sklearn.metrics import fbeta_score, make_scorer
ftwo_scorer = make_scorer(fbeta_score, beta=0.333)

In [181]:
def evaluateLogistic(C, k,
                    max_depth,
                    #min_child_weight,
                    #gamma,
                    #subsample,
                    #colsample_bytree,
                    #reg_alpha,
                    #n_estimators
                    ):
    
    model = LogisticRegression(C=C, solver='liblinear', random_state=42)
                     
    
    #estimator=RandomForestClassifier(max_depth=3, n_estimators=150, random_state=231),
    #estimator=LinearSVC(penalty="l1", dual=False),
    
    rf = RandomForestClassifier(max_depth=int(max_depth), n_estimators=300, random_state=231,
                               class_weight='balanced')
    
    #xgb = XGBClassifier(learning_rate =0.1,
     #                     n_estimators=300,
      #                    max_depth=int(max_depth),
       #                   min_child_weight=1,
        #                  gamma=gamma,
         #                 subsample=subsample,
          #                colsample_bytree=colsample_bytree,
           #               reg_alpha=reg_alpha,
            #              #reg_lambda=0.5123,
             #             objective= 'binary:logistic',
              #            n_jobs=-1,
               #           scale_pos_weight=1,
                #          seed=231)
    
    test_pipeline = Pipeline([
        ('feature_selection', SelectFromModel(estimator=rf,
                                             max_features=int(k),
                                             threshold=-np.inf)),
        ('classification', model),
    ])
    
    return np.mean(cross_validate(test_pipeline, X_train_trust1_prepared, y=y_train_trust1, cv=cv,
                              #scoring=profit_scoring,
                              n_jobs=-1,
                              scoring=ftwo_scorer,
                                  #scoring="f1"
                                 )['test_score'])

In [182]:
params_logistic = {
    'C': (1,500),
    'k': (2,20),
    'max_depth':(2,5.5),
    #'min_child_weight':(1,7),
    #'gamma': (0,1),
    #'subsample':(0.6,1),
    #'colsample_bytree':(0.6,1),
    #'reg_alpha':(0.001, 100),
    #'n_estimators': (50,300),
    #'reg_lambda':(0.3, 0.7)
}

In [183]:
optimization_logistic = BayesianOptimization(evaluateLogistic, params_logistic, random_state=231)
optimization_logistic.maximize(n_iter=200, init_points=75)

|   iter    |  target   |     C     |     k     | max_depth |
-------------------------------------------------------------
|  1        |  0.9449   |  391.0    |  12.52    |  3.493    |
|  2        |  0.9636   |  158.4    |  16.88    |  5.162    |
|  3        |  0.9562   |  20.43    |  18.47    |  2.212    |
|  4        |  0.8515   |  91.18    |  3.301    |  3.606    |
|  5        |  0.9056   |  354.5    |  10.7     |  2.089    |
|  6        |  0.9191   |  34.38    |  9.666    |  2.767    |
|  7        |  0.854    |  405.3    |  4.521    |  4.629    |
|  8        |  0.9562   |  140.3    |  19.08    |  5.098    |
|  9        |  0.9458   |  305.5    |  16.02    |  2.145    |
|  10       |  0.9498   |  393.4    |  18.24    |  3.515    |
|  11       |  0.9357   |  443.4    |  11.87    |  5.454    |
|  12       |  0.9498   |  283.1    |  18.22    |  3.973    |
|  13       |  0.9464   |  48.85    |  13.12    |  2.21     |
|  14       |  0.9562   |  290.6    |  17.69    |  4.799    |
|  15   

|  81       |  0.9498   |  321.9    |  20.0     |  5.5      |
|  82       |  0.9468   |  238.3    |  20.0     |  2.0      |
|  83       |  0.9562   |  377.9    |  19.92    |  2.024    |
|  84       |  0.9562   |  118.6    |  19.97    |  5.281    |
|  85       |  0.9562   |  222.0    |  19.8     |  5.483    |
|  86       |  0.8352   |  235.8    |  2.0      |  5.5      |
|  87       |  0.9562   |  264.7    |  19.85    |  2.162    |
|  88       |  0.9562   |  381.8    |  19.01    |  5.493    |
|  89       |  0.9562   |  213.6    |  19.96    |  2.049    |
|  90       |  0.9498   |  268.1    |  20.0     |  5.5      |
|  91       |  0.9562   |  77.47    |  19.87    |  5.486    |
|  92       |  0.8352   |  189.3    |  2.085    |  5.297    |
|  93       |  0.9213   |  1.0      |  16.49    |  5.5      |
|  94       |  0.9562   |  347.3    |  19.44    |  5.425    |
|  95       |  0.9562   |  494.7    |  19.46    |  2.013    |
|  96       |  0.9562   |  416.5    |  19.99    |  5.495    |
|  97   

KeyboardInterrupt: 

In [97]:
optimization_logistic.max

{'target': 340.0, 'params': {'C': 617.4371992275123, 'k': 8.026182029526707}}

### Show results of best performing model

In [184]:
select_from_model = Pipeline([
    ('feature_selection', SelectFromModel(estimator=RandomForestClassifier(max_depth=2,
                                                                           n_estimators=300, random_state=231,
                                                                           class_weight='balanced'),
                                             max_features=17,
                                             threshold=-np.inf)),
        ('classification', LogisticRegression(C=187.9, solver='liblinear', random_state=42)),
    ])

In [185]:
sum(cross_validate(select_from_model,
                   
                   X_train_trust1_prepared,
                   y=y_train_trust1,
                   cv=cv,
                   scoring=profit_scoring)['test_score'])

330

In [186]:
#...